# ChipChat Tutorial – Part I  
## Example: Binary to BCD Converter

In this notebook, we use the ChipChat workflow to generate and verify
a synthesizable Verilog module that converts a 5-bit binary number
(0–31) into packed BCD format.

We:
1. Use OpenAI via the same API flow shown in the ChipChat tutorial
2. Extract and clean the generated Verilog
3. Compile and simulate using **iverilog**
4. Verify correctness using the **unchanged testbench from the tutorial link**

In [1]:
!apt-get update
!apt-get install -y iverilog
!pip install --quiet openai

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,724 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,661 kB]
Get:

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

## OpenAI Setup

We configure the OpenAI API key exactly as shown in the ChipChat tutorial.
This is the only manual edit required to run the notebook.

In [3]:
from openai import OpenAI
client = OpenAI()

## Prompt Engineering

The following prompt precisely specifies:
- Module name
- Port names and widths (must match the testbench)
- Packed BCD encoding
- Combinational, synthesizable logic only

Being explicit here reduces LLM errors and minimizes cleanup.

In [6]:
prompt = """
Generate a synthesizable Verilog module named binary_to_bcd_converter.

Interface:
- input  [4:0] binary_input
- output [7:0] bcd_output

The output is packed BCD:
- bcd_output[3:0] = ones digit
- bcd_output[7:4] = tens digit

Valid inputs are 0 to 31.
Use purely combinational logic.
Do not use clocks, delays, initial blocks, or tasks.
The interface must exactly match the provided testbench.
"""
print(prompt)


Generate a synthesizable Verilog module named binary_to_bcd_converter.

Interface:
- input  [4:0] binary_input
- output [7:0] bcd_output

The output is packed BCD:
- bcd_output[3:0] = ones digit
- bcd_output[7:4] = tens digit

Valid inputs are 0 to 31.
Use purely combinational logic.
Do not use clocks, delays, initial blocks, or tasks.
The interface must exactly match the provided testbench.



## LLM Invocation

We now call the OpenAI API using the same pattern as the ChipChat tutorial
and capture the raw output for inspection.

## Verilog Cleanup

As demonstrated in the tutorial, we manually extract and clean the
LLM-generated Verilog to ensure:

- Synthesizability
- No extraneous constructs
- Exact match with the testbench interface

In [13]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

raw_output = response.choices[0].message.content
print(raw_output)

Here is a synthesizable Verilog module named `binary_to_bcd_converter` that converts a 5-bit binary input (ranging from 0 to 31) to a packed Binary-Coded Decimal (BCD) output. The output consists of the ones digit in the lower 4 bits and the tens digit in the upper 4 bits. The implementation uses purely combinational logic.

```verilog
module binary_to_bcd_converter (
    input  [4:0] binary_input,
    output [7:0] bcd_output
);

    reg [3:0] tens;
    reg [3:0] ones;

    always @(*) begin
        // Initialize the BCD digits
        tens = 4'b0000;
        ones = 4'b0000;

        // Convert binary to BCD
        if (binary_input < 10) begin
            ones = binary_input; // Directly assign for values 0-9
        end else if (binary_input < 20) begin
            tens = 4'b0001; // Set tens digit to 1
            ones = binary_input - 10; // Subtract 10 for ones digit
        end else begin
            tens = 4'b0010; // Set tens digit to 2
            ones = binary_input - 20; // 

In [14]:
verilog_code = """
`timescale 1ns / 1ps

module binary_to_bcd_converter (
    input  wire [4:0] binary_input,
    output reg  [7:0] bcd_output
);

    always @(*) begin
        bcd_output[3:0] = binary_input % 10;
        bcd_output[7:4] = binary_input / 10;
    end

endmodule
"""

In [15]:
with open("binary_to_bcd_converter.v", "w") as f:
    f.write(verilog_code)

## Testbench Download

We download the **unchanged testbench directly from the ChipChat tutorial
repository**, as required by the assignment.

In [17]:
!wget https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v -O binary_to_bcd_tb.v

--2026-02-10 03:40:01--  https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1078 (1.1K) [text/plain]
Saving to: ‘binary_to_bcd_tb.v’

binary_to_bcd_tb.v  100%[===================>]   1.05K  --.-KB/s    in 0s      

2026-02-10 03:40:01 (45.9 MB/s) - ‘binary_to_bcd_tb.v’ saved [1078/1078]



## Compilation and Simulation

We compile using `iverilog -g2012` and run the simulation with `vvp`.
A successful run prints **"All test cases passed!"**.

In [19]:
!iverilog -g2012 binary_to_bcd_converter.v binary_to_bcd_tb.v
!vvp a.out

Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!
